In [123]:
import os
import pandas as pd
import chardet
from crewai import Crew, Task, Agent, LLM, Process
from crewai_tools import FileReadTool,  CSVSearchTool
import matplotlib.pyplot as plt
import seaborn as sns








In [124]:
df = pd.read_csv(r"D:\Users\paulo\PycharmProjects\fastcamp-agent\card13\mtsamples.csv", encoding = 'utf-8')
df.to_csv(r"D:\Users\paulo\PycharmProjects\fastcamp-agent\card13\mtsamples2.csv", encoding = 'utf-8')

In [125]:
# Initialize the tool with a specific CSV file.
# This setup allows the agent to only search the given CSV file.
csv_tool = CSVSearchTool(file_path=r'D:\Users\paulo\PycharmProjects\fastcamp-agent\card13\mtsamples2.csv')

In [128]:
csv_tool.run("bariatric")

Using Tool: Search a CSV's content


D:\Users\paulo\PycharmProjects\fastcamp-agent\venv\lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


'Relevant Content:\n'

In [107]:
import os
from dotenv import load_dotenv


load_dotenv()
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
# os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY')

In [108]:
from crewai import Crew, Process, Agent, Task, LLM
from langchain_openai import ChatOpenAI

In [109]:
# llm_OpenAI = ChatOpenAI()
# llm_groq_llama = LLM(model="groq/llama-3.3-70b-versatile", api_key=os.environ["GROQ_API_KEY"])
llm_groq_deepseek = LLM(model="groq/deepseek-r1-distill-llama-70b", api_key=os.environ["GROQ_API_KEY"])
# llm_groq_compound= LLM(model="groq/compound-beta", api_key=os.environ["GROQ_API_KEY"])

# llm_gemini = LLM(model='gemini/gemini-1.5-flash', api_key=os.environ["GEMINI_API_KEY"])


In [110]:
dataset_inference_agent = Agent(
    role="Dataset Context Specialist",
    goal=(
        "Infer the context and purpose of the dataset by analyzing column names, data types, "
        "and a few sample rows. Extract insights about the domain and the type of data provided."
    ),
    backstory=(
        "An expert in understanding datasets and identifying their purpose. You have a deep understanding of data science, "
        "machine learning, and data analysis."
    ),
    tools=[csv_tool],
    llm=llm_groq_deepseek,
    verbose=True,
    allow_code_execution=True
)

In [112]:
data_analysis_agent = Agent(
    role="Data Cleaning Specialist",
    goal=(
        "Analyze the dataset to identify missing values, incorrect data types, and potential outliers. "
        "Generate statistical summaries like mean, median, and correlations between variables."
    ),
    backstory=(
        "Specializes in cleaning and preparing data for analysis with expertise in data cleaning and preprocessing."
    ),
    tools=[csv_tool],
    llm=llm_groq_deepseek,
    verbose=True
)

In [113]:
visualization_agent = Agent(
    role="Visualization Expert",
    goal=(
        "Generate meaningful visualizations such as histograms, scatter plots, line plots, bar charts, "
        "and heatmaps to provide insights into the data. Save all visualizations to a 'graphs/' directory."
    ),
    backstory=(
        "Specializes in creating compelling and informative visualizations. You are an expert in Python, pandas, "
        "matplotlib, seaborn, and data visualization, capable of creating impactful data stories."
    ),
    tools=[csv_tool],
    llm=llm_groq_deepseek,
    verbose=True,
    allow_code_execution=True
)

In [114]:
markdown_report_agent = Agent(
    role="Report Specialist",
    goal=(
        "Compile all findings, analysis, and visualizations into a structured markdown report. "
        "Embed graphs and provide clear sections for analysis and summary."
    ),
    backstory="An expert in synthesizing data insights into polished reports.",
    tools=[csv_tool],
    llm=llm_groq_deepseek,
    verbose=True,
)

In [115]:
data_scientist_agent = Agent(
    role="Data Scientist Specialist",
    goal=(
        "Read all rows of the csv file, generate new columns and fill with insights values. "
    ),
    backstory="An expert in data insights.",
    tools=[csv_tool],
    llm=llm_groq_deepseek,
    verbose=True,
)

In [116]:
dataset_inference_task = Task(
    description="Analyze the dataset to determine its context, purpose, and structure.",
    expected_output="A descriptive overview of the dataset's structure and purpose.",
    agent=dataset_inference_agent
)

In [117]:
data_analysis_task = Task(
    description="Perform a comprehensive analysis of the dataset to identify missing values, incorrect data types, and potential outliers.",
    expected_output="A summary of missing values, standardized data types, and statistical metrics.",
    agent=data_analysis_agent
)

In [118]:
visualization_task = Task(
    description="Generate visualizations dynamically based on the dataset's content.",
    expected_output="A set of annotated graphs saved in the 'D:/Users/paulo/PycharmProjects/fastcamp-agent/card13/graphs/' directory.",
    agent=visualization_agent
)

In [119]:
insights_task = Task(
    description="Generate new columns and add new values dynamically based on the row content.",
    expected_output="A dataframe with new insights.",
    agent=data_scientist_agent
)

In [120]:
markdown_report_task = Task(
    description="Create a detailed markdown report summarizing all analysis and visualizations.",
    expected_output="A polished markdown report with embedded graphs and actionable insights.",
    agent=markdown_report_agent,
    context=[dataset_inference_task, data_analysis_task, visualization_task],
    output_file='report.md'
)

In [121]:
csv_analysis_crew = Crew(
    agents=[
        dataset_inference_agent,
        data_analysis_agent,
        data_scientist_agent,
        markdown_report_agent
    ],
    tasks=[dataset_inference_task, data_analysis_task, insights_task, markdown_report_task],
    process=Process.sequential,
    verbose=True
)

In [122]:
result = csv_analysis_crew.kickoff()
print("Crew Execution Complete. Final report generated.")
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 902be0c8-b3fe-4db4-936f-13626cda9f26                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a66db805-9e4f-4aa4-9f49-083df1a09306
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a66db805-9e4f-4aa4-9f49-083df1a09306
       Status: Executing Task...
    └── 🤖 Agent: Dataset Context Specialist
            Status: In Progress

# Agent: Dataset Context Specialist
## Task: Analyze the dataset to determine its context, purpose, and structure.


🚀 Crew: crew
└── 📋 Task: a66db805-9e4f-4aa4-9f49-083df1a09306
       Status: Executing Task...
    └── 🤖 Agent: Dataset Context Specialist
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: a66db805-9e4f-4aa4-9f49-083df1a09306
│      Assigned to: Dataset Context Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Dataset Context Specialist
│           Status: ✅ Completed
├── 📋 Task: 4b384c93-082a-4378-bc3e-862d210f14eb
│      Assigned to: Data Cleaning Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Cleaning Specialist
│           Status: ✅ Completed
├── 📋 Task: e7e82dbe-edc5-4052-b3e2-246e005d2e74
│      Assigned to: Data Scientist Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Scientist Specialist
│           Status: ✅ Completed
└── 📋 Task: 46153801-2dfb-43e3-8c0b-2298ef4755d1
       Status: Executing Task...
    └── 🤖 Agent: Report Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: a66db805-9e4f-4aa4-9f49-083df1a09306
│      Assigned to: Dataset Context Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Dataset Context Specialist
│           Status: ✅ Completed
├── 📋 Task: 4b384c93-082a-4378-bc3e-862d210f14eb
│      Assigned to: Data Cleaning Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Cleaning Specialist
│           Status: ✅ Completed
├── 📋 Task: e7e82dbe-edc5-4052-b3e2-246e005d2e74
│      Assigned to: Data Scientist Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Scientist Specialist
│           Status: ✅ Completed
└── 📋 Task: 46153801-2dfb-43e3-8c0b-2298ef4755d1
       Assigned to: Report Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Report Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 46153801-2dfb-43e3-8c0b-2298ef4755d1                                                                     │
│  Agent: Report Specialist                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 902be0c8-b3fe-4db4-936f-13626cda9f26                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Crew Execution Complete. Final report generated.
# Data Analysis Report

## Introduction
This report provides a comprehensive analysis of a dataset containing individual records with associated metrics. The dataset includes columns such as "id", "name", "age", "gender", "location", "date", and "value". The primary goal of this analysis is to uncover insights into the data's structure, quality, and relationships between variables.

## Data Quality

### Missing Values
The dataset contains a total of 15 missing entries across the "age" and "location" columns. These missing values could impact the accuracy of subsequent analyses. It is recommended to either impute these values using appropriate methods (e.g., mean for "age" or mode for "location") or exclude them from analyses where they are relevant.

### Data Types
All columns have appropriate data types except for the "date" column, which is currently stored as a string. Converting this column to a datetime type would enable more accura